In [7]:
import requests
import json
import pandas as pd
from collections import OrderedDict

#import fbbt from web
#fbbt = requests.get("http://purl.obolibrary.org/obo/fbbt/2018-10-03/fbbt.json").json()
#fbbt['graphs'][0]['edges']

#import fbbt from /oort
fbbt = json.load(open("/Users/clare/git/drosophila-anatomy-developmental-ontology/oort/fbbt.json", "r"))
#fbbt['graphs'][0]['edges']

#compartments
c_infile = pd.read_csv("lb_compartments.tsv", sep="\t")
c_outfile = 'compartment_candidates.tsv'

#lineages
l_infile = pd.read_csv("lb_lineages.tsv", sep="\t")
l_outfile = 'lineage_candidates.tsv'

#fascicles
f_infile = pd.read_csv("lb_fascicles.tsv", sep="\t")
f_outfile = 'fascicle_candidates.tsv'


infile = c_infile
outfile = c_outfile
term_list = sorted(list(set(infile.larvalbrain_Name)))

#term_list
fbbt['graphs'][0]['nodes']


[{'id': 'http://purl.obolibrary.org/obo/FBbt_00007250',
  'meta': {'definition': {'val': 'Cell which is part of the boundary between ommatidia.',
    'xrefs': ['FBC:DOS']},
   'comments': ['The boundary between ommatidia is only a single cell wide, so each interommatidial cell is not part of any one repeating unit of organization of the retina. However, in some sources, they are included as part of ommatidia, with the repeating unit referred to as the ommatidial core.'],
   'xrefs': [{'val': 'VFB:FBbt_00007250'}],
   'synonyms': [{'pred': 'hasExactSynonym',
     'val': 'inter-ommatidial cell',
     'xrefs': []}],
   'basicPropertyValues': [{'pred': 'http://www.geneontology.org/formats/oboInOwl#hasOBONamespace',
     'val': 'fly_anatomy.ontology'},
    {'pred': 'http://www.geneontology.org/formats/oboInOwl#created_by',
     'val': 'djs93'},
    {'pred': 'http://www.geneontology.org/formats/oboInOwl#creation_date',
     'val': '2008-10-31T05:54:23Z'}]},
  'type': 'CLASS',
  'lbl': 'inter

In [2]:
def search_function(search_term: str):
    output_id=list()
    output_label=list()
    for n in fbbt['graphs'][0]['nodes']:
        try:
            test_result = True
            for word in str.lower(search_term).split(" "):
                if word not in str.lower(n['lbl']): test_result = False
                else: continue
            if test_result == True:
                output_id.append(n['id'])
                output_label.append(n['lbl'])
        except KeyError: continue
        try:
            for s in n['meta']['synonyms']:
                test_result = True
                for word in str.lower(search_term).split(" "):
                    if word not in str.lower(s['val']) or n['id'] in output_id: test_result = False
                    else: continue
                if test_result == True:
                    output_id.append(n['id'])
                    output_label.append(n['lbl'])
        except KeyError: continue
    return dict(zip(output_id,output_label))



In [3]:
mapping = dict()
for term in term_list:
    mapping[term] = search_function(term)
          

In [4]:
mapping

{'Abdominal ganglion': {'http://purl.obolibrary.org/obo/FBbt_00110173': 'adult abdominal neuromere',
  'http://purl.obolibrary.org/obo/FBbt_00111082': 'adult thoracico-abdominal ganglion cell body rind',
  'http://purl.obolibrary.org/obo/FBbt_00110053': 'larval Leucokinin ABLK neuron of the abdominal ganglion',
  'http://purl.obolibrary.org/obo/FBbt_00110051': 'adult Tachykinin neuron of the thoracic-abdominal ganglion',
  'http://purl.obolibrary.org/obo/FBbt_00110048': 'larval Tachykinin neuron of the ventral cord',
  'http://purl.obolibrary.org/obo/FBbt_00110054': 'adult Leucokinin ABLK neuron of the abdominal ganglion',
  'http://purl.obolibrary.org/obo/FBbt_00111304': 'adult sex peptide abdominal ganglion neuron',
  'http://purl.obolibrary.org/obo/FBbt_00111305': 'adult dorsal posterior Ilp7 (male) neuron',
  'http://purl.obolibrary.org/obo/FBbt_00004052': 'thoracico-abdominal ganglion',
  'http://purl.obolibrary.org/obo/FBbt_00047528': 'neuron projection bundle of thoracico-abdomi

In [281]:
map_table = pd.DataFrame(columns=('larvalbrain_Name','FBbt_id','FBbt_Name'))
row_od = OrderedDict([]) #new template row as an empty ordered dictionary

for key1 in mapping:
    if mapping[key1] == {}:
        row_od['larvalbrain_Name'] = key1
        row_od['FBbt_id'] = ""
        row_od['FBbt_Name'] = ""
        new_row = pd.DataFrame.from_records([row_od])
        map_table = pd.concat([map_table, new_row], ignore_index=True, sort=False)
    else:
        for key2 in mapping[key1]:
            row_od['larvalbrain_Name'] = key1
            row_od['FBbt_id'] = key2
            row_od['FBbt_Name'] = mapping[key1][key2]
            new_row = pd.DataFrame.from_records([row_od])
            map_table = pd.concat([map_table, new_row], ignore_index=True, sort=False)
    
#map_table

In [282]:
map_table.to_csv(outfile, sep = '\t')